In [15]:
from constraint import *
import pandas as pd
import numpy as np
import math

In [16]:
network1 = pd.read_csv('edgesGRADIS.csv')
network2 = pd.read_csv('edgesTIGRESS.csv')
network3 = pd.read_csv('edgesGENIE.csv')
networks = [network3, network2, network1]

In [17]:
def get_longest_dictionary(dicts):
    sizes = []
    for dic in dicts:
        sizes.append(len(dic))
    max_size = max(sizes)
    return sizes.index(max_size)

In [18]:
#network['gxy'] = network['gx'].astype(str) + network['gy'].astype(str)
def my_todict(network, str1, str2): # str1 = gxy, str2 = value
    key = list(network[str1])
    value = list(network[str2])
    mydict = {}
    for i in range(len(key)):
        mydict[key[i]] = value[i]
    return mydict

In [19]:
def calculate_domain (networks, gx, gy, edge_weight, step = 0.1): 
    # step is float, gx, gy, edge_weight are strings (labels of network csv file)
    factors = []
    networks_dicts = []
    for network in networks:
        network['gxy'] = network[gx].astype(str) + network[gy].astype(str)
        network_dict = my_todict(network, 'gxy', edge_weight)
        networks_dicts.append(network_dict)
    longest_dict_index = get_longest_dictionary(networks_dicts)
    for key in networks_dicts[longest_dict_index]:
        myvalues = []
        for i in range (len(networks_dicts)):
            try :
                myvalues.append(networks_dicts[i][key])
            except KeyError:
                continue
        factors.append(myvalues)
    means = []
    std_dev = []
    for lst in factors:
        means.append(sum(lst)/len(lst))
        std_dev.append(np.std(lst))
    domain = {}
    keys = list(networks_dicts[longest_dict_index].keys())
    for i in range(len(networks_dicts[longest_dict_index])):
        key = keys[i]
        #domain[key] = np.arange(means[i]-(std_dev[i]/2), means[i]+(std_dev[i]/2)+0.1, step)
        domain[key] = np.asarray([means[i]-(std_dev[i]/2), means[i], means[i]+(std_dev[i]/2)])

    return domain, keys, means

In [20]:
domains, keys, means = calculate_domain ([network2, network3], 'gx', 'gy', 'value')
print(domains)
len(domains)

{'G1G5': array([0.648931, 0.763154, 0.877377]), 'G5G1': array([0.499344, 0.499344, 0.499344]), 'G6G8': array([0.4987405, 0.611027 , 0.7233135]), 'G8G6': array([0.366752, 0.366752, 0.366752]), 'G9G10': array([0.397001, 0.470934, 0.544867]), 'G2G8': array([0.249629, 0.249629, 0.249629]), 'G9G7': array([0.236426, 0.236426, 0.236426]), 'G7G4': array([0.207432, 0.207432, 0.207432]), 'G10G9': array([0.204197, 0.204197, 0.204197]), 'G2G6': array([0.26359825, 0.3246655 , 0.38573275]), 'G7G9': array([0.201392, 0.201392, 0.201392]), 'G5G9': array([0.198182, 0.198182, 0.198182]), 'G3G4': array([0.22220575, 0.2486705 , 0.27513525]), 'G6G2': array([0.194773, 0.194773, 0.194773]), 'G4G3': array([0.187063, 0.187063, 0.187063]), 'G7G2': array([0.2678875, 0.358725 , 0.4495625]), 'G3G7': array([0.2915285, 0.418219 , 0.5449095]), 'G8G2': array([0.160942, 0.160942, 0.160942]), 'G7G3': array([0.2267555, 0.303437 , 0.3801185]), 'G4G10': array([0.144416, 0.144416, 0.144416]), 'G2G1': array([0.133893, 0.13389

90

In [220]:
#network.info()

In [21]:
def sparsity_constraint(Regulator_Values):
    ascending_order = sorted(Regulator_Values)
    lower_limit = np.mean(ascending_order[0: int(len(Regulator_Values)/3)])
    descending_order = np.flip(sorted(Regulator_Values))
    upper_limit = np.mean(descending_order[0:int(len(Regulator_Values)/3)])
    return (lower_limit, upper_limit)

In [22]:
def redundant_edge (problem, networks, gx, gy, edge_weight, limits):
    domains, keys, means = calculate_domain(networks, gx, gy, edge_weight)
    lower_limit = limits[0]
    upper_limit = limits[1]
    reverse_key_set = set()
    for i in range(len(keys)):
        
        key = keys[i]
        if key in reverse_key_set:
            continue

        max_domain = max(domains[key])

        reverse_key = get_reverse_gene(key)
        
        mean_key = np.mean(domains[key])
        try:
            mean_reverse_key = np.mean(domains[reverse_key])
        except KeyError:
            mean_reverse_key = 0
            
        if mean_reverse_key > mean_key:
            primary_key = reverse_key
            secondary_key = key
        else :
            primary_key = key
            secondary_key = reverse_key
        
        mean_index = keys.index(primary_key)
        mean = means[mean_index]
        
        reverse_key_set.add(secondary_key)
        try:
            max_secondary = max(domains[secondary_key])
        except KeyError:
            max_secondary = 0
        
        try:
            print('added ' + primary_key)
            problem.addVariable(primary_key, domains[primary_key])
            #problem.addConstraint(lambda x : lower_limit < x and x < upper_limit, [primary_key])
            #problem.addConstraint(lambda x : x > mean and max_secondary < x/2, [primary_key])
        except ValueError:
            continue

        if max_secondary > mean:
            try :
                print('added inverse ' + secondary_key)
                problem.addVariable(secondary_key, domains[secondary_key])
            except ValueError :
                continue
    problem.addConstraint(boundary_constraint)    
    return problem

In [23]:
def get_reverse_gene(gene):
    second_g_index = gene[1:len(gene)].index('G') + 1
    part1 = gene[1:second_g_index]
    part2 = gene[second_g_index: len(gene)]
    return part2 + 'G' + part1

In [24]:
def boundary_constraint(*arr):
    limits = sparsity_constraint(means)
    lower_limit = limits[0]
    upper_limit = limits[1]
    result = True
    for v in arr:
        if v < lower_limit or v > upper_limit :
            result = False
    return result

In [25]:
limits = sparsity_constraint(means)
instance = Problem()
grn = redundant_edge(instance, networks, 'gx', 'gy', 'value', limits)
#grn.getSolutions()

added G1G5
added G8G6
added inverse G6G8
added G8G6
added G9G10
added G2G8
added inverse G8G2
added G9G7
added G7G4
added G6G2
added G5G9
added G4G3
added G6G2
added G4G3
added G7G2
added G3G7
added inverse G7G3
added G10G4
added G2G1
added inverse G1G2
added G9G3
added G5G4
added G5G10
added G5G2
added G4G1
added G5G2
added G10G4
added G3G2
added G1G3
added G9G4
added G10G3
added G7G1
added inverse G1G7
added G8G3
added G1G8
added G5G6
added G5G3
added G7G8
added G5G7
added G7G10
added G6G3
added G10G3
added G3G2
added G1G8
added G9G2
added G10G1
added inverse G1G10
added G7G6
added G8G5
added G6G1
added inverse G1G6
added G8G9
added G6G4
added G9G2
added G8G4
added G6G10
added G9G1
added G10G2
added G8G10
added G7G1
added G6G9
added G4G1
added G4G2
added G9G1
added G10G1
added G6G1
added G4G2


In [26]:
solution = grn.getSolutions()

KeyboardInterrupt: 